In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [61]:
# 步骤1：得到图像分块的embedding
def image2embedding_naive(image, patch_size, weight):
    """
    通过分块的方法对图片embedding，将图像按照patch大小进行分块，将每一块编码成一个model_dim长的向量
    :param image: Batch * C * H * W
    :param patch_size: 每一个块的大小
    :param weight: 线性变换的权重，将二维图像映射到embedding的维度
    :return: img_patch_embedding # bs * block_num * model_dim
    """
    # F.unfold 可以取出当前的区域块所有channel flatten后的结果，可以用来分块
    # shape: torch.Size([1, 4, 48])
    patch = F.unfold(image, patch_size, stride=patch_size).transpose(-1, -2)
    # 将patch与weight相乘，使用@
    # torch.Size([1, 4, 48]) @ torch.Size([48 * 8]) = torch.Size([1, 4, 8])
    img_patch_embedding = patch @ weight
    return img_patch_embedding


In [62]:
def image2embedding_conv(img, kernel, stride):
    """
    用卷积的方法实现image embedding，将图像按照patch大小进行分块，将每一块编码成一个model_dim长的向量
    :param img: shape: Batch * C * H * W
    :param kernel: shape: output_channel * input_channel * patch_size * patch_size
    :param stride: patch_size
    :return: img_patch_embedding: shape: # bs * block_num * model_dim
    """
    # 通过卷积得到每一个块的向量，每一个块的embedding矩阵的深度方向
    conv_output = F.conv2d(img, kernel, stride=stride)  # 输出的shape: batch * output_channel * oh * ow
    # 将每个块变成一个向量
    bs, oc, oh, ow = conv_output.shape  # oc就是model_dim
    img_patch_embedding = conv_output.reshape(bs, oc, oh*ow).transpose(-1, -2) # bs * block_num * model_dim
    return img_patch_embedding


In [63]:
# 生成一张图
bs, input_channel, img_h, img_w = 1, 3, 8, 8
img = torch.randn(bs, input_channel, img_h, img_w)
# 定义patch_size, model_dim
patch_size = 4  # 将4*4的区域看作一个块
model_dim = 8
# 定义weight 和 对应的卷积kernel形式
weight = torch.randn(patch_size*patch_size*3, 8)
kernel = weight.transpose(0,1).reshape((model_dim, input_channel, patch_size, patch_size))

# 得到输出
# 维度：bs * block_num * model_dim = 1 * 4 * 8
img_patch_embedding_naive = image2embedding_naive(img, patch_size, weight)
img_patch_embedding_conv = image2embedding_conv(img, kernel, patch_size)

print(img_patch_embedding_naive)
print(img_patch_embedding_conv)

# 输出结果完全一致，得到结论：这两种方式是完全等价的！ naive的实现更容易理解，卷积的实现方式更加优雅

tensor([[[  4.2487,   5.8423,  -4.2135,  -2.9166,  -9.9933,   1.5447,   7.3950,
            9.6445],
         [  6.1628,   1.7814,  -0.0337,  -7.0407,   6.4813,  -5.6805,   7.9561,
          -10.4764],
         [  4.9292,  -6.8243,  -8.1931, -12.8321,   3.7562,   1.7118,   1.5872,
           -0.7664],
         [ -8.3185,  -2.3029,  -7.4757,  -6.8871,  -3.9029,  -4.6566,  -2.7030,
           -7.3441]]])
tensor([[[  4.2487,   5.8423,  -4.2135,  -2.9166,  -9.9933,   1.5447,   7.3950,
            9.6445],
         [  6.1628,   1.7813,  -0.0337,  -7.0407,   6.4813,  -5.6805,   7.9561,
          -10.4764],
         [  4.9292,  -6.8243,  -8.1931, -12.8321,   3.7562,   1.7118,   1.5872,
           -0.7664],
         [ -8.3185,  -2.3029,  -7.4757,  -6.8871,  -3.9029,  -4.6566,  -2.7030,
           -7.3441]]])


In [64]:
# 步骤2：在分块的embedding的向量前面加一个类别编码，随机初始化，可学习的
cls_embedding = torch.randn(bs, 1, model_dim,requires_grad=True)
token_embedding = torch.cat([cls_embedding, img_patch_embedding_naive], dim=1)
print(token_embedding)
# token_embedding shape: # bs * (block_num+1) * model_dim

tensor([[[ -0.2357,   0.1158,  -0.1739,   0.7241,   0.7202,   0.1991,  -0.6002,
           -0.1781],
         [  4.2487,   5.8423,  -4.2135,  -2.9166,  -9.9933,   1.5447,   7.3950,
            9.6445],
         [  6.1628,   1.7814,  -0.0337,  -7.0407,   6.4813,  -5.6805,   7.9561,
          -10.4764],
         [  4.9292,  -6.8243,  -8.1931, -12.8321,   3.7562,   1.7118,   1.5872,
           -0.7664],
         [ -8.3185,  -2.3029,  -7.4757,  -6.8871,  -3.9029,  -4.6566,  -2.7030,
           -7.3441]]], grad_fn=<CatBackward0>)


In [65]:
# 步骤3：增加位置编码，随机初始化的向量，可学习的，与token_embedding利用广播机制相加
# 作者原文试用了比较多不同的位置编码，发现效果也都差不多
# position_embedding shape: (block_num+1) * model_dim
# token_embedding shape: bs * (block_num+1) * model_dim
position_embedding = torch.randn(token_embedding.shape[1], model_dim)
token_embedding = token_embedding + position_embedding
print(token_embedding)

tensor([[[ -0.4270,   0.7457,   0.8264,  -0.6647,  -1.2300,   0.4762,   0.2305,
            0.2514],
         [  4.3150,   5.2303,  -4.8529,  -2.8020, -10.6923,   0.8593,   9.2035,
           10.0592],
         [  4.1246,   1.9462,  -0.3134,  -6.1022,   5.2276,  -5.5611,   7.2997,
           -8.2624],
         [  5.5617,  -7.2962,  -7.0161, -13.5277,   4.4145,   0.8199,   2.2333,
            0.5186],
         [ -8.6957,  -2.4894,  -7.6328,  -7.3801,  -3.3299,  -3.4335,  -2.1942,
          -10.3034]]], grad_fn=<AddBackward0>)


In [66]:
# 步骤4：将token_embedding送入transformer的encoder api
# encoder_output shape: bs * (block_num+1) * model_dim
encoder_layer = nn.TransformerEncoderLayer(d_model=8, nhead=4)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
encoder_output = transformer_encoder(token_embedding)

print(encoder_output)

tensor([[[ 0.4872,  0.3004,  1.2018,  1.6626, -1.1272, -0.8182, -0.6702,
          -1.0364],
         [-1.4297, -0.0746, -0.3529, -1.5762,  0.3194,  1.0503,  1.0893,
           0.9743],
         [-0.8310, -0.9227,  0.1259, -1.8030,  1.0001,  0.4261,  0.9869,
           1.0178],
         [-0.8473, -0.7808, -0.0186, -1.8803,  0.8297,  0.7777,  0.9795,
           0.9401],
         [-1.1098, -1.0349,  0.9337, -1.6075,  1.0043,  0.7980,  0.7078,
           0.3084]]], grad_fn=<NativeLayerNormBackward0>)


In [67]:
num_classes =10
gt = torch.empty(bs, dtype=torch.long).random_(10)  # shape: bs 值：0-9十个类别中的一个
# 步骤5：取出经过encoder后cls位置的向量，映射到类别，与标签做损失
encoder_output_cls = encoder_output[:, 0, :]
linear_layer = nn.Linear(model_dim, num_classes)
cls_logits = linear_layer(encoder_output_cls)  # shape: bs * num_classes
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(cls_logits, gt)
print(loss)


tensor(2.0730, grad_fn=<NllLossBackward0>)
